In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
data_path = r'C:\Users\cheng\Desktop\20240201\Question\Wimbledon_featured_matches.csv'
data = pd.read_csv(data_path)
print(data.head())

              match_id         player1        player2 elapsed_time  set_no  \
0  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:00       1   
1  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:38       1   
2  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:01       1   
3  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:31       1   
4  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:02:21       1   

   game_no  point_no  p1_sets  p2_sets  p1_games  ...  p2_break_pt_won  \
0        1         1        0        0         0  ...                0   
1        1         2        0        0         0  ...                0   
2        1         3        0        0         0  ...                0   
3        1         4        0        0         0  ...                0   
4        1         5        0        0         0  ...                0   

  p1_break_pt_missed p2_break_pt_missed  p1_distance_run  p2_distance_run  \
0        

In [31]:
# 按match_id分组
grouped = data.groupby('match_id')
print(grouped.size())

match_id
2023-wimbledon-1301    300
2023-wimbledon-1302    201
2023-wimbledon-1303    134
2023-wimbledon-1304    337
2023-wimbledon-1305    246
2023-wimbledon-1306    332
2023-wimbledon-1307    232
2023-wimbledon-1308    190
2023-wimbledon-1309    213
2023-wimbledon-1310    318
2023-wimbledon-1311    170
2023-wimbledon-1312    275
2023-wimbledon-1313    290
2023-wimbledon-1314    185
2023-wimbledon-1315    198
2023-wimbledon-1316    167
2023-wimbledon-1401    225
2023-wimbledon-1402    274
2023-wimbledon-1403    122
2023-wimbledon-1404    286
2023-wimbledon-1405    215
2023-wimbledon-1406    195
2023-wimbledon-1407    331
2023-wimbledon-1408    271
2023-wimbledon-1501    189
2023-wimbledon-1502    283
2023-wimbledon-1503    193
2023-wimbledon-1504    217
2023-wimbledon-1601    159
2023-wimbledon-1602    202
2023-wimbledon-1701    334
dtype: int64


In [32]:
sample_match = grouped.get_group('2023-wimbledon-1301')
print(sample_match)

                match_id         player1        player2 elapsed_time  set_no  \
0    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:00       1   
1    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:38       1   
2    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:01       1   
3    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:31       1   
4    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:02:21       1   
..                   ...             ...            ...          ...     ...   
295  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:53:25       4   
296  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:54:01       4   
297  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:02       4   
298  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:34       4   
299  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:56:11       4   

     game_no  point_no  p1_sets  p2_set

In [34]:
def analyse_match(match):
    match_id = match['match_id']
    player1 = match['player1']
    player2 = match['player2']
    # point_victor,winner of the point,"1 if player 1 wins, 2 if player 2 wins"
    # 统计单盘比赛中每个球员的得分情况
    set_grouped = match.groupby('set_no')
    res_dict = {'player1_score':[], 'player2_score':[], 
                'player1_win_double_count':[], 'player2_win_double_count':[], 'player1_loss_after_win':[], 'player2_loss_after_win':[],
                'player1_win_with_serve':[], 'player2_win_with_serve':[], 'player1_loss_with_serve':[], 'player2_loss_with_serve':[],
                'player1_win_double_count_with_serve':[], 'player2_win_double_count_with_serve':[], 'player1_loss_after_win_with_serve':[], 'player2_loss_after_win_with_serve':[],
                'player1_win_double_count_with_return':[], 'player2_win_double_count_with_return':[], 'player1_loss_after_win_with_return':[], 'player2_loss_after_win_with_return':[]
                }
    for set_no, set_data in set_grouped:
        player1_score = 0
        player2_score = 0
        # 统计 player 连赢两盘的次数
        player1_win_double_count = 0
        player2_win_double_count = 0
        # 统计 player 连赢一盘后输掉下一盘的次数\
        player1_loss_after_win = 0
        player2_loss_after_win = 0
        # 统计 player 赢得发球局的次数
        player1_win_with_serve = 0
        player2_win_with_serve = 0
        # 统计 player 输掉接发局的次数
        player1_loss_with_serve = 0
        player2_loss_with_serve = 0
        # 统计 player 赢得发球局且连赢两次的次数
        player1_win_double_count_with_serve = 0
        player2_win_double_count_with_serve = 0
        player1_loss_after_win_with_serve = 0
        player2_loss_after_win_with_serve = 0
        # 统计 player 输掉接发局且连赢两次的次数
        player1_win_double_count_with_return = 0
        player2_win_double_count_with_return = 0
        player1_loss_after_win_with_return = 0
        player2_loss_after_win_with_return = 0

        flag = -1
        pre_game_no = 0
        for index, row in set_data.iterrows():
            point_victor = row['point_victor']
            serve_flag = row['server']
            if point_victor == 1:
                player1_score += 1
                player1_win_double_count += (flag == 1)
                player2_loss_after_win += (flag == 2)
                if serve_flag == 1:
                    player1_win_with_serve += 1
                    player1_win_double_count_with_serve += (flag == 1)
                    player2_loss_after_win_with_return += (flag == 2)
                else:
                    player2_loss_with_serve += 1
                    player1_win_double_count_with_return += (flag == 1)
                    player2_loss_after_win_with_serve += (flag == 2)
            else:
                player2_score += 1
                player2_win_double_count += (flag == 2)
                player1_loss_after_win += (flag == 1)
                if serve_flag == 2:
                    player2_win_with_serve += 1
                    player2_win_double_count_with_serve += (flag == 2)
                    player1_loss_after_win_with_return += (flag == 1)
                else:
                    player1_loss_with_serve += 1
                    player2_win_double_count_with_return += (flag == 2)
                    player1_loss_after_win_with_serve += (flag == 1)
            flag = point_victor
            # 检测是否是新的局
            game_no = row['game_no']
            if game_no != pre_game_no:
                flag = -1
                pre_game_no = game_no

        res_dict['player1_score'].append(player1_score)
        res_dict['player2_score'].append(player2_score)
        res_dict['player1_win_double_count'].append(player1_win_double_count)
        res_dict['player2_win_double_count'].append(player2_win_double_count)
        res_dict['player1_loss_after_win'].append(player1_loss_after_win)
        res_dict['player2_loss_after_win'].append(player2_loss_after_win)
        res_dict['player1_win_with_serve'].append(player1_win_with_serve)
        res_dict['player2_win_with_serve'].append(player2_win_with_serve)
        res_dict['player1_loss_with_serve'].append(player1_loss_with_serve)
        res_dict['player2_loss_with_serve'].append(player2_loss_with_serve)
        res_dict['player1_win_double_count_with_serve'].append(player1_win_double_count_with_serve)
        res_dict['player2_win_double_count_with_serve'].append(player2_win_double_count_with_serve)
        res_dict['player1_loss_after_win_with_serve'].append(player1_loss_after_win_with_serve)
        res_dict['player2_loss_after_win_with_serve'].append(player2_loss_after_win_with_serve)
        res_dict['player1_win_double_count_with_return'].append(player1_win_double_count_with_return)
        res_dict['player2_win_double_count_with_return'].append(player2_win_double_count_with_return)
        res_dict['player1_loss_after_win_with_return'].append(player1_loss_after_win_with_return)
        res_dict['player2_loss_after_win_with_return'].append(player2_loss_after_win_with_return)
        
    # print(res_dict)
    return res_dict
res_dict = analyse_match(sample_match)

In [35]:
# 卡方检验Momentum
def chi_square_test_momentum(res_dict):
    player_score = res_dict['player1_score'] + res_dict['player2_score']
    player_loss = res_dict['player2_score'] + res_dict['player1_score']
    player_win_double_count = res_dict['player1_win_double_count'] + res_dict['player2_win_double_count']
    player_loss_after_win = res_dict['player1_loss_after_win'] + res_dict['player2_loss_after_win']
    # print(player1_score)

    # 卡方检验
    a = sum(player_score) 
    b = sum(player_win_double_count)
    c = sum(player_loss)
    d = sum(player_loss_after_win)
    chi_square_score = (a*d-b*c)**2*(a+b+c+d)/(a+b)/(c+d)/(a+c)/(b+d)
    # print(chi_square_score)
    return chi_square_score

# 卡方检验发球优势
def chi_square_test_serve(res_dict):
    player_score = res_dict['player1_score'] + res_dict['player2_score']
    player_loss = res_dict['player2_score'] + res_dict['player1_score']
    player_win_with_serve = res_dict['player1_win_with_serve'] + res_dict['player2_win_with_serve']
    player_loss_with_serve = res_dict['player1_loss_with_serve'] + res_dict['player2_loss_with_serve']
    # print(player1_score)

    # 卡方检验
    a = sum(player_score)
    b = sum(player_win_with_serve)
    c = sum(player_loss)
    d = sum(player_loss_with_serve)
    chi_square_score = (a*d-b*c)**2*(a+b+c+d)/(a+b)/(c+d)/(a+c)/(b+d)
    # print(chi_square_score)
    return chi_square_score

# remove 发球优势对检验momentum的影响
def chi_square_test_momentum_without_serve(res_dict):
    player_score = res_dict['player1_win_with_serve' ] + res_dict['player2_win_with_serve']
    player_loss = res_dict['player1_loss_with_serve'] + res_dict['player2_loss_with_serve']
    player_win_double_count_with_serve = res_dict['player1_win_double_count_with_serve'] + res_dict['player2_win_double_count_with_serve']
    player_loss_after_win_with_serve = res_dict['player1_loss_after_win_with_serve'] + res_dict['player2_loss_after_win_with_serve']
    # print(player1_score)

    # 卡方检验
    a = sum(player_score)
    b = sum(player_win_double_count_with_serve)
    c = sum(player_loss)
    d = sum(player_loss_after_win_with_serve)
    
    chi_square_score = (a*d-b*c)**2*(a+b+c+d)/(a+b)/(c+d)/(a+c)/(b+d)
    # print(chi_square_score)
    return chi_square_score

In [36]:
# 检验momentum
# 穷举所有的比赛
chi_square_score_list = []
for match_id, match in grouped:
    res_dict = analyse_match(match)
    chi_square_score = chi_square_test_momentum(res_dict)
    chi_square_score_list.append(chi_square_score)
print(chi_square_score_list)
# 穷举所有比赛的结果求和
match_all = {'player1_score':[], 'player2_score':[], 'player1_win_double_count':[], 'player2_win_double_count':[], 'player1_loss_after_win':[], 'player2_loss_after_win':[]}
for match_id, match in grouped:
    res_dict = analyse_match(match)
    for key in match_all.keys():
        match_all[key].extend(res_dict[key])
print(match_all)
chi_square_score_all = chi_square_test_momentum(match_all)
print(chi_square_score_all)


[0.002780240048468091, 0.35365942085262125, 0.9744670255567396, 0.3039437362889045, 0.08718599883731602, 1.2630155016283067, 2.528251464179369, 1.4988575630638319, 0.20146150199303636, 2.93329089564871, 2.49998982964658, 0.5211916127307715, 0.0725066639174468, 0.671861411625561, 1.4208333333333334, 0.005275235227953904, 0.31034112416943366, 0.012531240058369924, 0.11063475682670729, 0.1984801199549493, 1.5681702687352406, 3.829975832861979, 1.928630271858907, 7.211506043260018, 1.0476954645901293, 0.519694829190294, 4.19042258051546, 0.10033312653299228, 1.2422291855431389, 0.03813144677911194, 0.06205282014569693]
{'player1_score': [34, 44, 34, 46, 22, 34, 43, 20, 18, 13, 26, 34, 32, 29, 48, 27, 32, 34, 36, 29, 36, 36, 47, 28, 41, 36, 46, 26, 42, 22, 25, 27, 24, 39, 34, 41, 39, 23, 26, 34, 25, 8, 36, 32, 41, 38, 32, 44, 49, 30, 34, 28, 45, 38, 27, 27, 20, 13, 37, 27, 31, 35, 29, 40, 37, 37, 20, 34, 36, 19, 36, 16, 30, 36, 44, 45, 31, 21, 33, 29, 27, 39, 31, 39, 34, 30, 35, 39, 34, 21,

In [37]:
# 检验发球优势
chi_square_score_list = []
for match_id, match in grouped:
    res_dict = analyse_match(match)
    chi_square_score = chi_square_test_serve(res_dict)
    chi_square_score_list.append(chi_square_score)
print(chi_square_score_list)
# 穷举所有比赛的结果求和
match_all = {'player1_score':[], 'player2_score':[], 'player1_win_with_serve':[], 'player2_win_with_serve':[], 'player1_loss_with_serve':[], 'player2_loss_with_serve':[]}
for match_id, match in grouped:
    res_dict = analyse_match(match)
    for key in match_all.keys():
        match_all[key].extend(res_dict[key])
print(match_all)
chi_square_score_all = chi_square_test_serve(match_all)
print(chi_square_score_all)


[18.181818181818183, 37.710588115172555, 12.63324617860016, 23.7798760690529, 16.674064171122993, 38.98945498945499, 37.004204530042045, 26.55538197943963, 15.975000000000001, 4.8424475552206845, 10.715318031360239, 4.494634459674473, 12.680244442551741, 15.436965108624094, 19.771474672967212, 16.09887787816963, 15.976491019240678, 31.09801248605121, 5.638732585081094, 29.74697216771234, 17.68092105263158, 14.625, 49.59715142241773, 50.42444407008086, 22.44992513830381, 25.36508718769009, 12.997460183397683, 15.673373770178621, 5.164071337669683, 18.346174259177356, 10.990081399820536]
{'player1_score': [34, 44, 34, 46, 22, 34, 43, 20, 18, 13, 26, 34, 32, 29, 48, 27, 32, 34, 36, 29, 36, 36, 47, 28, 41, 36, 46, 26, 42, 22, 25, 27, 24, 39, 34, 41, 39, 23, 26, 34, 25, 8, 36, 32, 41, 38, 32, 44, 49, 30, 34, 28, 45, 38, 27, 27, 20, 13, 37, 27, 31, 35, 29, 40, 37, 37, 20, 34, 36, 19, 36, 16, 30, 36, 44, 45, 31, 21, 33, 29, 27, 39, 31, 39, 34, 30, 35, 39, 34, 21, 45, 27, 31, 36, 16, 23, 42, 3

In [40]:
# remove 发球优势对检验momentum的影响
# chi_square_score_list = []
# for match_id, match in grouped:
#     res_dict = analyse_match(match)
#     chi_square_score = chi_square_test_momentum_without_serve(res_dict)
#     chi_square_score_list.append(chi_square_score)
# print(chi_square_score_list)
# 穷举所有比赛的结果求和
match_all = {'player1_win_with_serve':[], 'player2_win_with_serve':[], 'player1_loss_with_serve':[], 'player2_loss_with_serve':[], 'player1_win_double_count_with_serve':[], 'player2_win_double_count_with_serve':[], 'player1_loss_after_win_with_serve':[], 'player2_loss_after_win_with_serve':[]}
for match_id, match in grouped:
    res_dict = analyse_match(match)
    for key in match_all.keys():
        match_all[key].extend(res_dict[key])
print(match_all)
chi_square_score_all = chi_square_test_momentum_without_serve(match_all)
print(chi_square_score_all)


{'player1_win_with_serve': [26, 31, 20, 25, 13, 28, 29, 15, 13, 11, 15, 23, 20, 20, 33, 20, 23, 22, 21, 18, 28, 26, 34, 16, 28, 29, 30, 18, 26, 18, 16, 16, 20, 19, 17, 27, 29, 12, 16, 25, 18, 5, 20, 18, 24, 26, 16, 34, 28, 17, 24, 12, 28, 29, 18, 20, 15, 11, 29, 14, 23, 17, 22, 32, 28, 26, 14, 20, 16, 14, 32, 15, 20, 20, 31, 18, 20, 15, 25, 12, 20, 27, 21, 30, 28, 20, 30, 31, 24, 20, 29, 21, 20, 21, 9, 18, 28, 16, 20, 11, 16, 18, 23, 6, 19, 15, 22, 18, 14, 14, 19, 28, 9, 31, 13, 20, 21], 'player2_win_with_serve': [15, 34, 17, 27, 23, 30, 30, 17, 20, 16, 21, 15, 18, 22, 36, 21, 12, 20, 23, 11, 26, 26, 37, 13, 29, 27, 29, 24, 32, 20, 22, 12, 14, 23, 11, 22, 24, 18, 7, 28, 23, 12, 18, 15, 15, 23, 16, 30, 28, 13, 18, 12, 31, 27, 21, 22, 20, 12, 28, 24, 13, 23, 13, 14, 29, 30, 20, 23, 18, 16, 28, 16, 19, 19, 30, 29, 17, 20, 13, 11, 14, 22, 16, 31, 30, 18, 31, 28, 22, 20, 28, 18, 18, 22, 18, 20, 30, 11, 19, 16, 15, 12, 19, 19, 27, 16, 14, 17, 12, 23, 23, 31, 18, 31, 23, 18, 20], 'player1_los